## Stabilization P = 1.4

In [1]:
#-----Import some essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Some important variables

p_max = 1.4;
t_max = 5.0;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_5.0_P_1.4_ramp.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [2]:
#----Integrate using NPT

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max, tau = 3.5, tauP = 9.5, P = p_max);
npt.randomize_velocities(seed = 42)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_max) + "_P_" + str(p_max) + "_stabilization.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_max) + "_stabilization.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_max) + "_stabilization.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 

In [3]:
hoomd.run(1e5)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 756152 / 856000 | TPS 15.1576 | ETA 01:49:47
Time 00:00:20 | Step 756317 / 856000 | TPS 16.369 | ETA 01:41:29
Time 00:00:30 | Step 756500 / 856000 | TPS 18.1744 | ETA 01:31:14
Time 00:00:40 | Step 756692 / 856000 | TPS 19.174 | ETA 01:26:19
Time 00:00:50 | Step 756881 / 856000 | TPS 18.7937 | ETA 01:27:54
Time 00:01:00 | Step 757058 / 856000 | TPS 17.6944 | ETA 01:33:11
Time 00:01:10 | Step 757221 / 856000 | TPS 16.2505 | ETA 01:41:18
Time 00:01:20 | Step 757397 / 856000 | TPS 17.5758 | ETA 01:33:30
Time 00:01:30 | Step 757581 / 856000 | TPS 18.2423 | ETA 01:29:55
Time 00:01:40 | Step 757770 / 856000 | TPS 18.84 | ETA 01:26:53
Time 00:01:50 | Step 757948 / 856000 | TPS 17.7941 | ETA 01:31:50
Time 00:02:00 | Step 758123 / 85600

In [4]:
npt.set_params(tau = 4.0);

In [5]:
hoomd.run(1e5)

** starting run **
Time 01:33:41 | Step 856177 / 956000 | TPS 17.651 | ETA 01:34:15
Time 01:33:52 | Step 856355 / 956000 | TPS 17.6729 | ETA 01:33:58
Time 01:34:02 | Step 856531 / 956000 | TPS 17.5513 | ETA 01:34:27
Time 01:34:12 | Step 856707 / 956000 | TPS 17.5918 | ETA 01:34:04
Time 01:34:22 | Step 856884 / 956000 | TPS 17.6668 | ETA 01:33:30
Time 01:34:32 | Step 857061 / 956000 | TPS 17.5804 | ETA 01:33:47
Time 01:34:42 | Step 857240 / 956000 | TPS 17.8688 | ETA 01:32:06
Time 01:34:52 | Step 857417 / 956000 | TPS 17.6908 | ETA 01:32:52
Time 01:35:02 | Step 857596 / 956000 | TPS 17.8024 | ETA 01:32:07
Time 01:35:12 | Step 857774 / 956000 | TPS 17.7068 | ETA 01:32:27
Time 01:35:22 | Step 857951 / 956000 | TPS 17.682 | ETA 01:32:25
Time 01:35:32 | Step 858129 / 956000 | TPS 17.7126 | ETA 01:32:05
Time 01:35:42 | Step 858310 / 956000 | TPS 17.9552 | ETA 01:30:40
Time 01:35:52 | Step 858489 / 956000 | TPS 17.7782 | ETA 01:31:24
Time 01:36:02 | Step 858660 / 956000 | TPS 16.9989 | ETA 01

**ERROR**: nlist.tree(): Particle 803 is out of bounds (x: 46.0942, y: -25.9277, z: 0.122138, fx: 2.51982, fy: -0.636131, fz:0.505352)


RuntimeError: Error updating neighborlist

In [ ]:
npt.set_params(tau = 4.5);

In [ ]:
hoomd.run(1e5)

In [ ]:
npt.set_params(tau = 5.0);

In [ ]:
hoomd.run(1e5)

In [ ]:
npt.set_params(tau = 5.5);

In [ ]:
hoomd.run(1e5)